<a href="https://colab.research.google.com/github/vitorsanp/dsprojects/blob/SandboxProjects/SandBox_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#libraries basic
import os
import time
import re

#others
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

#lightgbm
import lightgbm as lgb

In [ ]:
###STE 0.3 DATA LOAD
#%%time
TARGET_NAME = 'target'

train_data = pd.read_csv('/content/drive/My Drive/Colab datasets/AutoML/train.csv')
train_data[TARGET_NAME] = train_data[TARGET_NAME].str.slice(start=6).astype(int) - 1
train_data.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,...,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,1,0,4,1,1,0,0,2,0,0,0,0,0,1,0,0,0,0,...,0,0,2,0,0,11,0,1,1,0,1,0,7,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,6,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,1,0,0,1,6,1,5,0,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,7,2,2,0,0,0,58,0,10,0,0,0,0,0,3,0,...,1,0,0,0,0,0,0,0,0,0,2,1,5,0,0,4,0,0,2,1,0,1,0,0,1,1,2,2,0,22,0,1,2,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,3,0,0,0,0,0,0,0,4,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0


In [ ]:
test_data = pd.read_csv('/content/drive/My Drive/Colab datasets/AutoML/test.csv')
test_data.head(1)

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,...,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
0,1,0,0,0,0,0,0,0,0,0,3,0,0,0,3,2,1,0,0,0,0,0,0,0,5,3,1,1,0,0,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,11,1,20,0,0,0,0,0


In [ ]:
submission = pd.read_csv('/content/drive/My Drive/Colab datasets/AutoML/sampleSubmission.csv')
submission.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,1,0,0,0,0,0,0,0,0
1,2,1,0,0,0,0,0,0,0,0
2,3,1,0,0,0,0,0,0,0,0
3,4,1,0,0,0,0,0,0,0,0
4,5,1,0,0,0,0,0,0,0,0


In [ ]:
###Step 0.5 Add new features

def create_gr_feats(data):
  pass

all_df = pd.concat([train_data, test_data]).reset_index(drop = True)
create_gr_feats(all_df)
train_data, test_data = all_df[:len(train_data)], all_df[len(train_data):]
print(train_data.shape, test_data.shape)

(61878, 95) (144368, 95)


#LIGHTGBM PRESET USAGE

In [ ]:
#!pip install scikit-optimize

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.neighbors import KNeighborsClassifier

from sklearn.feature_selection import SelectKBest

In [ ]:
remove = ['id', TARGET_NAME]

X = train_data.drop(remove, axis=1)
y = train_data[TARGET_NAME]

Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size = 0.5, random_state = 0)

model = lgb.LGBMClassifier(n_estimators=100,
                           learning_rate=0.05,
                           max_depth=6,
                           subsample=0.8,
                           colsample_bytree=0.8,
                           random_state=9)
model.fit(Xtrain, ytrain)

res = dict()
for col in X.columns:
  Xval_ = Xval.copy()
  Xval_[col] = np.random.permutation(Xval_[col])

  p = model.predict_proba(Xval_)
  ll = log_loss(yval, p)
  res[col] = ll
  print("Feature: {} | LL: {:.4f} | LL Delta: {:.4f}".format(col, ll, 1 - ll))

Feature: feat_1 | LL: 0.6369 | LL Delta: 0.3631
Feature: feat_2 | LL: 0.6343 | LL Delta: 0.3657
Feature: feat_3 | LL: 0.6363 | LL Delta: 0.3637
Feature: feat_4 | LL: 0.6351 | LL Delta: 0.3649
Feature: feat_5 | LL: 0.6340 | LL Delta: 0.3660
Feature: feat_6 | LL: 0.6339 | LL Delta: 0.3661
Feature: feat_7 | LL: 0.6363 | LL Delta: 0.3637
Feature: feat_8 | LL: 0.6405 | LL Delta: 0.3595
Feature: feat_9 | LL: 0.6582 | LL Delta: 0.3418
Feature: feat_10 | LL: 0.6343 | LL Delta: 0.3657
Feature: feat_11 | LL: 0.7984 | LL Delta: 0.2016
Feature: feat_12 | LL: 0.6339 | LL Delta: 0.3661
Feature: feat_13 | LL: 0.6355 | LL Delta: 0.3645
Feature: feat_14 | LL: 0.6755 | LL Delta: 0.3245
Feature: feat_15 | LL: 0.6762 | LL Delta: 0.3238
Feature: feat_16 | LL: 0.6401 | LL Delta: 0.3599
Feature: feat_17 | LL: 0.6475 | LL Delta: 0.3525
Feature: feat_18 | LL: 0.6352 | LL Delta: 0.3648
Feature: feat_19 | LL: 0.6364 | LL Delta: 0.3636
Feature: feat_20 | LL: 0.6349 | LL Delta: 0.3651
Feature: feat_21 | LL: 0.6340

In [ ]:
fimp_perm = (1 - pd.Series(res)).sort_values() * -1

In [ ]:
fimp_lgbm = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)

In [ ]:
fimps = pd.DataFrame({"fimp_perm": fimp_perm, "fimp_lgbm": fimp_lgbm})#.corr()
fimps.corr()

,fimp_perm,fimp_lgbm
fimp_perm,1.00000,0.58508
fimp_lgbm,0.58508,1.00000


In [ ]:
fimps_rank = fimps.rank(ascending=False).sort_values("fimp_perm")

In [ ]:
fimps_rank.mean(axis=1).sort_values().head(20)

feat_60     3.0
feat_40     4.5
feat_67     4.5
feat_15     5.0
feat_34     6.5
feat_14     7.0
feat_42     7.5
feat_11     9.0
feat_25    10.0
feat_43    10.5
feat_86    10.5
feat_9     12.5
feat_62    13.5
feat_75    14.0
feat_24    14.5
feat_26    15.0
feat_48    18.0
feat_39    21.5
feat_36    22.0
feat_88    22.5
dtype: float64

In [ ]:
%%time

Xtest = test_data.drop(remove, axis=1)
test_pred = model.predict_proba(Xtest)
print("Prediction for test data:\n{}\nShape = {}".format(test_pred[:10], test_pred.shape))

Prediction for test data:
[[3.74647080e-03 3.30963587e-01 1.94745449e-01 4.00645669e-01
  2.47479330e-03 2.51093446e-02 1.95746620e-02 1.40913130e-02
  8.64871115e-03]
 [3.08583491e-02 1.60216826e-01 6.80704663e-02 2.03639915e-02
  7.29336246e-03 5.83763283e-01 1.55318149e-02 9.67701679e-02
  1.71317395e-02]
 [1.92989444e-03 5.22330718e-03 3.44205267e-03 2.34429347e-03
  7.12354682e-04 9.61788911e-01 2.41414428e-03 1.90663332e-02
  3.07870890e-03]
 [8.01388138e-03 4.84241330e-01 3.46338029e-01 5.93486120e-02
  3.62197307e-03 2.73119778e-02 8.15009362e-03 1.79258629e-02
  4.50482407e-02]
 [5.12714208e-02 1.62607160e-02 1.04403103e-02 3.65038637e-03
  4.17121649e-03 2.46314965e-02 1.33430752e-02 2.36576355e-01
  6.39655023e-01]
 [4.48816448e-03 2.81255830e-01 6.56918132e-01 3.69394354e-03
  2.38563388e-03 2.15930133e-02 7.22775914e-03 1.58740308e-02
  6.56349280e-03]
 [8.22947241e-03 2.02011580e-02 1.34001557e-02 1.81803402e-02
  1.39613840e-03 1.71511925e-02 5.61962481e-02 8.54135801e-0

In [ ]:
## Step 5 Submission

In [ ]:
submission.head(10)

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,1,0,0,0,0,0,0,0,0
1,2,1,0,0,0,0,0,0,0,0
2,3,1,0,0,0,0,0,0,0,0
3,4,1,0,0,0,0,0,0,0,0
4,5,1,0,0,0,0,0,0,0,0
5,6,1,0,0,0,0,0,0,0,0
6,7,1,0,0,0,0,0,0,0,0
7,8,1,0,0,0,0,0,0,0,0
8,9,1,0,0,0,0,0,0,0,0
9,10,1,0,0,0,0,0,0,0,0


In [ ]:
test_pred

array([[0.00374647, 0.33096359, 0.19474545, ..., 0.01957466, 0.01409131,
        0.00864871],
       [0.03085835, 0.16021683, 0.06807047, ..., 0.01553181, 0.09677017,
        0.01713174],
       [0.00192989, 0.00522331, 0.00344205, ..., 0.00241414, 0.01906633,
        0.00307871],
       ...,
       [0.00547298, 0.63395259, 0.14978793, ..., 0.01862916, 0.02792321,
        0.01478242],
       [0.00237083, 0.462746  , 0.04389554, ..., 0.00424378, 0.00864436,
        0.00542544],
       [0.00529692, 0.36470144, 0.45776956, ..., 0.05950797, 0.0180181 ,
        0.00957988]])

In [ ]:
submission.iloc[:,1:] = test_pred
submission.to_csv('otto_prediction_v4.csv', index = False)

In [ ]:
submission.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,0.003746,0.330964,0.194745,0.400646,0.002475,0.025109,0.019575,0.014091,0.008649
1,2,0.030858,0.160217,0.068070,0.020364,0.007293,0.583763,0.015532,0.096770,0.017132
2,3,0.001930,0.005223,0.003442,0.002344,0.000712,0.961789,0.002414,0.019066,0.003079
3,4,0.008014,0.484241,0.346338,0.059349,0.003622,0.027312,0.008150,0.017926,0.045048
4,5,0.051271,0.016261,0.010440,0.003650,0.004171,0.024631,0.013343,0.236576,0.639655
